## Check if we have dbatools docker images

In [ ]:
docker images "dbatools/*"

## Download dbatools docker images

In [ ]:
# get the base images
docker pull dbatools/sqlinstance
docker pull dbatools/sqlinstance2

## Verify if we have containers setup

In [ ]:
docker ps -a

### If containers are not up and running, start them

In [ ]:
# start container dockersql1
docker start dockersql1

# start container dockersql2 
docker start dockersql2

### If not available we need to start them with parameters

In [ ]:
# stop and remove containers
docker kill dockersql1, dockersql2
docker rm dockersql1, dockersql2

# setup two containers and expose ports
docker run `
    -p 1433:1433 `
    -p 5022:5022 `
    --network localnet `
    -v sharedFolder:/tmp `
    --hostname dockersql1 `
    --name dockersql1 `
    -d dbatools/sqlinstance
    

docker run `
    -p 14333:1433 `
    -p 5023:5022 `
    --network localnet `
    -v sharedFolder:/tmp `
    --hostname dockersql2 `
    --name dockersql2 `
    -d dbatools/sqlinstance2

## Set variables values

In [ ]:
$dbatools1 = "localhost,1433"
$dbatools2 = "localhost,14333"
$dbatoolsEdge = "raspberrypi.lan"
$login = "sqladmin"
$loginEdge = "sa"
$database = "Pubs"
$AG = "test-ag"
$edgeDatabase = "EmptyNorthwind"
$edgeCentralDB = "CentralDB"

# dbatools docker images
$secureString = ConvertTo-SecureString "dbatools.IO" -AsPlainText -Force
$cred = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $login, $secureString

$secureStringEdge = ConvertTo-SecureString "myStrong!Password" -AsPlainText -Force
$credEdge = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $loginEdge, $secureStringEdge

# Azure SQL Edge

### Verify if Azure SQL Server on Edge has our login with proper permissions

In [ ]:
# Create 
New-DbaLogin -SqlInstance $dbatoolsEdge -SqlCredential $credEdge -Login $login -SecurePassword $secureString

# Add login to sysadmin server role
Add-DbaServerRoleMember -SqlInstance $dbatoolsEdge -SqlCredential $credEdge -Login $login -ServerRole sysadmin -Confirm:$false

 Drop database from copy table data demo

In [ ]:
# Remove Database from Raspberry Pi
Remove-DbaDatabase -SqlInstance $dbatoolsEdge -SqlCredential $cred -Database $edgeDatabase -Confirm:$false

Drop sp\_whoIsActive from master database

In [ ]:
Invoke-DbaQuery -SqlInstance $dbatoolsEdge -SqlCredential $cred -Database "master" -Query "DROP PROCEDURE IF EXISTS dbo.sp_WhoisActive"

Create central DB with connection strings to all instances

In [ ]:
# Drop database CentralDB from Raspberry Pi
# Remove Database from Raspberry Pi
Remove-DbaDatabase -SqlInstance $dbatoolsEdge -SqlCredential $cred -Database $edgeCentralDB -Confirm:$false

# New database
New-DbaDatabase -SqlInstance $dbatoolsEdge -SqlCredential $cred -Name $edgeCentralDB

# New table and insert data
Invoke-DbaQuery -SqlInstance $dbatoolsEdge -SqlCredential $cred -File "D:\Presentations\GroupBy Conf Oct 2020\Reset\CentralDB.sql"

# Docker local laptop

## Remove 'test-ag' from AG

In [ ]:
# Remove Database from AG
Remove-DbaAgDatabase -SqlInstance $dbatools1 -SqlCredential $cred -AvailabilityGroup $AG -Database $database -Confirm:$false

# Remove Availability group
Remove-DbaAvailabilityGroup -SqlInstance $dbatools1, $dbatools2 -SqlCredential $cred -AvailabilityGroup $AG -Confirm:$false

# Remove Database from secondary instance
Remove-DbaDatabase -SqlInstance $dbatools2 -SqlCredential $cred -Database $database -Confirm:$false

# Grant permissions

In [ ]:
$filePermissions = "D:\Presentations\GroupBy Conf Oct 2020\Reset\02_GrantPermissions.sql"

$scriptWithoutGO = (Get-Content $filePermissions -Raw) -replace '\bGO\b', ' '

Invoke-DbaQuery -SqlInstance $dbatools1 -SqlCredential $cred -Query $scriptWithoutGO -Verbose

Drop `storageuserColleague`

In [ ]:
Remove-DbaLogin -SqlInstance $dbatools1 -SqlCredential $cred -Login 'storageuserColleague' -Confirm:$false
Remove-DbaDbUser -SqlInstance $dbatools1 -SqlCredential $cred -User 'storageuserColleague' -Confirm:$false

### Check Permissions

In [ ]:
Get-DbaUserPermission -SqlInstance $dbatools1 -SqlCredential $cred | Format-Table

### Remove files from directories

In [ ]:
Remove-Item "D:\Presentations\GroupBy Conf Oct 2020\Replicate Permissions\*.sql" -Confirm:$false

# Remove Export-DbaDatabase temp folder content

In [ ]:
Remove-Item "D:\Presentations\GroupBy Conf Oct 2020\Export-DbaDatabase\temp\*" -Recurse -Confirm:$false -Force

Remove-Item "D:\Presentations\GroupBy Conf Oct 2020\Export-DbaDatabase\*" -Exclude "temp" -Recurse -Confirm:$false -Force

<hr>

## Reset for copy data demo

Remove `EmptyNorthwind` database from `localhost,14333` instance

In [ ]:
Remove-DbaDatabase -SqlInstance $dbatools2 -SqlCredential $cred -Database EmptyNorthwind -Confirm:$false

Drop table `[dbo].[CopyOf_Order Details]`

In [ ]:
Invoke-DbaQuery -SqlInstance $dbatools1 -SqlCredential $cred -Database Northwind -Query "DROP TABLE [dbo].[CopyOf_Order Details]"

# Reset for refresh database demo
### Create `dbatools` database from `pubs` on `dockersql1` and also copy to `dockersql2`
### They will have different users

In [ ]:
# Remove database if exists
Remove-DbaDatabase -SqlInstance $dbatools1, $dbatools2 -SqlCredential $cred -Database "dbatools" -Confirm:$false

#Same instance
$copyDatabaseSplat = @{
    Source = $dbatools1
    SourceSqlCredential = $cred
    Destination = $dbatools1
    DestinationSqlCredential = $cred
    Database = "pubs"
    BackupRestore = $true
    SharedPath = "/tmp" 
    NewName = "dbatools"
}
Copy-DbaDatabase @copyDatabaseSplat

# Create login/user with "_qa"
$loginUser_qa = "dbatools_qa"
$refreshDatabase = "dbatools"
New-DbaLogin -SqlInstance $dbatools1 -SqlCredential $cred -Login $loginUser_qa -SecurePassword $secureString
New-DbaDbUser -SqlInstance $dbatools1 -SqlCredential $cred -Username $loginUser_qa -Login $loginUser_qa -Database $refreshDatabase

# Grant some permissions
$grants = "GRANT CREATE VIEW TO [dbatools_qa]  AS [dbo];
GRANT SELECT ON OBJECT::[dbo].[authors] TO [dbatools_qa]  AS [dbo];
GRANT DELETE ON OBJECT::[dbo].[authors] TO [dbatools_qa]  AS [dbo];
GRANT INSERT ON OBJECT::[dbo].[authors] TO [dbatools_qa]  AS [dbo];
REVOKE SELECT ON [dbo].[authors] TO [guest] AS [dbo]
"
# Run the exported script
Invoke-DbaQuery -SqlInstance $dbatools1 -SqlCredential $cred -Database $refreshDatabase -Query $grants -Verbose

$copyDatabaseSplat = @{
    Source = $dbatools1
    SourceSqlCredential = $cred
    Destination = $dbatools2
    DestinationSqlCredential = $cred
    Database = "dbatools"
    BackupRestore = $true
    SharedPath = "/tmp"
}
Copy-DbaDatabase @copyDatabaseSplat


# Create login/user with "_dev"
$loginUser_dev = "dbatools_dev"
$refreshDatabase = "dbatools"
New-DbaLogin -SqlInstance $dbatools2 -SqlCredential $cred -Login $loginUser_dev -SecurePassword $secureString -Force
New-DbaDbUser -SqlInstance $dbatools2 -SqlCredential $cred -Username $loginUser_dev -Login $loginUser_dev -Database $refreshDatabase

# Grant some permissions
$grants = "GRANT CREATE VIEW TO [dbatools_dev];
GRANT SELECT ON OBJECT::[dbo].[authors] TO [dbatools_dev];
"
# Run the exported script
Invoke-DbaQuery -SqlInstance $dbatools2 -SqlCredential $cred -Database $refreshDatabase -Query $grants -Verbose

Remove-DbaDbUser -SqlInstance $dbatools2 -SqlCredential $cred -Database $refreshDatabase -User $loginUser_qa

<hr>
Clean up xlsx files from `Test-Compliance` folder

In [ ]:
Remove-Item "D:\Presentations\GroupBy Conf Oct 2020\Test-Compliance\*.xlsx" -Recurse -Confirm:$false -Force

Clean up xlsx files from `Excel-Report` folder

In [ ]:
Remove-Item "D:\Presentations\GroupBy Conf Oct 2020\Excel-Report\*.xlsx" -Recurse -Confirm:$false -Force